## Description

This notebook is to investigate how different convergence properties related to each other. The goal is to find the best properties defined that are materials independent and can be used to predict the convergence of a PP. 

The properties that are investigated are:
- For pressure, compare the complex defined SSSP v1 residue volume and the vannila hydrostatic pressure
- For EOS metrics, compare nu wrt AE and nu with ref 200Ry. (Check and assure the guess that delta' and nu are correlated)
- Compare pressure and EOS metrics (nu ref 200Ry)
- Other pair see if those are correlated or not

What I think is, if I tuning the criteria of properties, there will be a cross from A > B to B > A. The different between if A, B are correlated or not is whether their will be a state where A, B are highly linearly correlated. 

The testing data is generated by running full convergence test in the grid of [20:5:200] Ry for all different properties calculation method, then can extract and construct the properties date from the output.
The tested PPs are Hg, Ga, N, Cs, Mn from gbrv, dojo, psl-paw-high and jth, in order to cover PPs from different generated code sources and different type of elements.

The AiiDA data is stored at group `SI/convergence-properties-compare`. 

In [1]:
from aiida import load_profile
import typing as t

load_profile("2023-08-07")

from aiida import orm

In [14]:
from aiida_sssp_workflow.workflows.convergence.pressure import helper_get_volume_from_pressure_birch_murnaghan
from aiida_sssp_workflow.calculations.calculate_bands_distance import get_bands_distance

def extract_data_scan_list1(node):
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergenceBandsWorkChain':
            lst = []
            for wf2 in wf.called:

                if wf2.process_label == 'helper_bands_distence_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']

        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}

    for i, wf in enumerate(lst):
        cutoff = scan_list[i]
        band_structure_a = wf.inputs.band_structure_a
        band_parameters_a = wf.inputs.band_parameters_a.get_dict()
        band_structure_b = wf.inputs.band_structure_b
        band_parameters_b = wf.inputs.band_parameters_b.get_dict()
        smearing = wf.inputs.smearing.value
        fermi_shift = wf.inputs.fermi_shift.value
        do_smearing = wf.inputs.do_smearing.value
        spin = wf.inputs.spin.value

        bandsdata_a = {
            "number_of_electrons": band_parameters_a["number_of_electrons"],
            "number_of_bands": band_parameters_a["number_of_bands"],
            "fermi_level": band_parameters_a["fermi_energy"],
            "bands": band_structure_a.get_bands(),
            "kpoints": band_structure_a.get_kpoints(),
            "weights": band_structure_a.get_array("weights"),
        }
        bandsdata_b = {
            "number_of_electrons": band_parameters_b["number_of_electrons"],
            "number_of_bands": band_parameters_b["number_of_bands"],
            "fermi_level": band_parameters_b["fermi_energy"],
            "bands": band_structure_b.get_bands(),
            "kpoints": band_structure_b.get_kpoints(),
            "weights": band_structure_b.get_array("weights"),
        }

        res = get_conv_data1(
            bandsdata_a,
            bandsdata_b,
            smearing,
            fermi_shift,
            do_smearing,
            spin,
        )
        data[cutoff] = res

    return data, scan_list

def get_conv_data1(bandsdata_a, bandsdata_b, smearing, fermi_shift, do_smearing, spin) -> float:
    res = get_bands_distance(
        bandsdata_a,
        bandsdata_b,
        smearing,
        fermi_shift,
        do_smearing,
        spin,
    )

    return res['eta_v']

def extract_data_scan_list2(node):
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergencePressureWorkChain':
            lst = []
            for wf2 in wf.called:

                if wf2.process_label == 'helper_pressure_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}

    for i, wf in enumerate(lst):
        cutoff = scan_list[i]

        i_para = wf.inputs.input_parameters.get_dict()
        r_para = wf.inputs.ref_parameters.get_dict()
        V0 = wf.inputs.V0.value
        B0 = wf.inputs.B0.value
        B1 = wf.inputs.B1.value
    
        # Get the data
        # Ref_200_nu: the nu value w.r.t. the 200 Ry reference
        # Ref_200_deltap: the delta_p value w.r.t. the 200 Ry reference
        res = get_conv_data2(i_para, r_para, V0, B0, B1)
        data[cutoff] = res

    return data, scan_list

def get_conv_data2(i_para, r_para, V0, B0, B1) -> float:
    """Return AE_delta (per atom), AE_nu, REL_nu (the nu compare to the reference not to AE)"""
    res_pressure = i_para["hydrostatic_stress"]
    ref_pressure = r_para["hydrostatic_stress"]
    absolute_diff = abs(res_pressure - ref_pressure)
    relative_diff = helper_get_volume_from_pressure_birch_murnaghan(
        absolute_diff, V0, B0, B1,
    )

    return relative_diff


In [15]:
g = 'SI/convergence-properties-compare'
gs_nodes = []
gs_nodes.extend(orm.Group.collection.get(label=g).nodes)
    
all_data1 = {}
all_data2 = {}
for node in gs_nodes:
    # give a node and the tuple of criteria
    # return the deducted cutoffs of A and B
    try:
        data1, scan_list1 = extract_data_scan_list1(node)
        data2, scan_list2 = extract_data_scan_list2(node)
        all_data1[node.pk] = {
            "data": data1,
            "scan_list": scan_list1    
        }
        all_data2[node.pk] = {
            "data": data2,
            "scan_list": scan_list2    
        }
    except Exception as e:
        #print(f"Error: {e}")
        #continue
        print(node.pk)
        raise e

In [16]:
def extract_cutoff(data, scan_list, criteria):
    """Extract the cutoff for pA and pB from a verification workchain

    Args:
        data (dict): the data extracted from the verification workchain
        scan_list (list): the list of cutoffs used in the verification workchain
        criteria (tuple): first element is the criteria for pA, second element is the criteria for pB

    Returns:
        tuple: the cutoff for pA and pB.
    """
    # Get the cutoff of pA and pB
    cut = 200
    for cutoff in reversed(scan_list):
        try: 
            p = data[cutoff]
        except:
            continue
        
        if p > criteria:
            break

        cut = cutoff

    return cut

In [17]:
def compute_cutoff(data12_tuple, criteria):
    cut_A_lst = []
    cut_B_lst = []

    all_data1 = data12_tuple[0]
    all_data2 = data12_tuple[1]
    criteria1 = criteria[0]
    criteria2 = criteria[1]

    for node_pk in all_data1:
        data = all_data1[node_pk]['data']
        scan_list = all_data1[node_pk]['scan_list']
        cut_A = extract_cutoff(data, scan_list, criteria1)
        cut_A_lst.append(cut_A)
    
    for node_pk in all_data2:
        data = all_data2[node_pk]['data']
        scan_list = all_data2[node_pk]['scan_list']
        cut_B = extract_cutoff(data, scan_list, criteria2)
        cut_B_lst.append(cut_B)
        
    return cut_A_lst, cut_B_lst

In [18]:
# Get data for plotting
cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(all_data1, all_data2), criteria=(0.1, 0.1))

In [19]:
import ipywidgets as ipw
import plotly.graph_objects as go

trace_corr_scatter = go.Scatter(x=cut_A_lst, y=cut_B_lst, mode='markers', name='cutoff correlation')
trace_xy_line = go.Scatter(x=[0, 200], y=[0, 200], name='x=y')
g = go.FigureWidget(data=[trace_corr_scatter, trace_xy_line])
g.layout.xaxis.title = 'cutoff pA'
g.layout.yaxis.title = 'cutoff pB'


In [26]:
pA_slider = ipw.FloatSlider(value=0.1, min=0.00, max=20.0, step=0.5, description='pA')
pB_slider = ipw.FloatSlider(value=0.1, min=0.00, max=1.0, step=0.01, description='pB')

def response(change):
    cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(all_data1, all_data2), criteria=(pA_slider.value, pB_slider.value))
    with g.batch_update():
        g.data[0].x = cut_A_lst
        g.data[0].y = cut_B_lst
        
pA_slider.observe(response, names="value")
pB_slider.observe(response, names="value")

slider_widgets = ipw.HBox([pA_slider, pB_slider])
app = ipw.VBox([slider_widgets, g])
app

In [36]:
# heatmap
import numpy as np

def compute_correlation(x, y):
    cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(all_data1, all_data2), criteria=(x, y))
    arr_A = np.array(cut_A_lst)
    arr_B = np.array(cut_B_lst)
    
    z = np.sum(np.abs(arr_A - arr_B) / 40.0)
    
    return z

xxs = np.linspace(0.0, 20, 201)
yys = np.linspace(0.0, 2, 201)
# get z map from xxs and yys
zzs = np.zeros((201, 201))
for i, x in enumerate(xxs):
    for j, y in enumerate(yys):
        zzs[i, j] = compute_correlation(x, y)
        print(f"x={x}, y={y}, z={zzs[i, j]}")
        
fig = go.FigureWidget(data=go.Heatmap(z=zzs, x=xxs, y=yys, zmax=30, zmin=0))
fig.layout.xaxis.title = 'cutoff from bands distance'
fig.layout.yaxis.title = 'cutoff from pressure difference'
fig

x=0.0, y=0.0, z=0.0
x=0.0, y=0.01, z=12.25
x=0.0, y=0.02, z=17.5
x=0.0, y=0.03, z=21.375
x=0.0, y=0.04, z=24.125
x=0.0, y=0.05, z=31.25
x=0.0, y=0.06, z=32.5
x=0.0, y=0.07, z=34.625
x=0.0, y=0.08, z=36.625
x=0.0, y=0.09, z=42.0
x=0.0, y=0.1, z=46.25
x=0.0, y=0.11, z=47.25
x=0.0, y=0.12, z=47.25
x=0.0, y=0.13, z=48.875
x=0.0, y=0.14, z=49.75
x=0.0, y=0.15, z=49.875
x=0.0, y=0.16, z=50.125
x=0.0, y=0.17, z=51.125
x=0.0, y=0.18, z=51.375
x=0.0, y=0.19, z=51.375
x=0.0, y=0.2, z=51.375
x=0.0, y=0.21, z=53.125
x=0.0, y=0.22, z=53.125
x=0.0, y=0.23, z=53.25
x=0.0, y=0.24, z=53.75
x=0.0, y=0.25, z=54.625
x=0.0, y=0.26, z=55.0
x=0.0, y=0.27, z=55.0
x=0.0, y=0.28, z=55.25
x=0.0, y=0.29, z=57.625
x=0.0, y=0.3, z=58.25
x=0.0, y=0.31, z=58.625
x=0.0, y=0.32, z=58.75
x=0.0, y=0.33, z=58.875
x=0.0, y=0.34, z=58.875
x=0.0, y=0.35000000000000003, z=59.375
x=0.0, y=0.36, z=59.375
x=0.0, y=0.37, z=59.5
x=0.0, y=0.38, z=59.75
x=0.0, y=0.39, z=60.25
x=0.0, y=0.4, z=60.25
x=0.0, y=0.41000000000000003, z=60.

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '4dfec21a-0af1-4cf9-96d7-61d04e828612',
              'x': array([ 0. ,  0.1,  0.2, ..., 19.8, 19.9, 20. ]),
              'y': array([0.  , 0.01, 0.02, ..., 1.98, 1.99, 2.  ]),
              'z': array([[ 0.  , 12.25, 17.5 , ..., 72.  , 72.  , 72.  ],
                          [46.5 , 34.25, 30.25, ..., 27.25, 27.25, 27.25],
                          [52.75, 40.5 , 36.  , ..., 21.75, 21.75, 21.75],
                          ...,
                          [81.5 , 69.25, 64.  , ...,  9.5 ,  9.5 ,  9.5 ],
                          [81.5 , 69.25, 64.  , ...,  9.5 ,  9.5 ,  9.5 ],
                          [81.5 , 69.25, 64.  , ...,  9.5 ,  9.5 ,  9.5 ]]),
              'zmax': 30,
              'zmin': 0}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'cutoff from bands distance'}},
               'yaxis': {'title': {'text': 'cutoff from pressure difference'}}}
})

In [34]:
zzs

array([[ 0.  , 12.25, 17.5 , ..., 72.  , 72.  , 72.  ],
       [46.5 , 34.25, 30.25, ..., 27.25, 27.25, 27.25],
       [52.75, 40.5 , 36.  , ..., 21.75, 21.75, 21.75],
       ...,
       [81.5 , 69.25, 64.  , ...,  9.5 ,  9.5 ,  9.5 ],
       [81.5 , 69.25, 64.  , ...,  9.5 ,  9.5 ,  9.5 ],
       [81.5 , 69.25, 64.  , ...,  9.5 ,  9.5 ,  9.5 ]])

In [35]:
compute_correlation(15, 0.14)

31.375